In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from src.models.arcface import iresnet100
import numpy as np
from PIL import Image

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = iresnet100()
state_dict = torch.load("checkpoints/arcface-r100-glint360k.pth", map_location=device)
model.load_state_dict(state_dict, strict=False)
model.eval()
model = model.to(device)

In [7]:
dummy_input = torch.randn(1, 3, 112, 112).to(device)

torch.onnx.export(
    model,
    dummy_input,
    "checkpoints/arcface-r100-glint360k.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11,
    do_constant_folding=True,
    dynamic_axes={"input": {0: "batch_size"}}
)

e:\Amir\Projects\face\src\models\arcface.py:192: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.fp16):


trtexec --onnx=arcface-r100-glint360k.onnx --saveEngine=arcface-r100-glint360k_fp16.engine --fp16 --workspace=2048


In [64]:
preprocess = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
])

def get_embedding(img_path):
    img = Image.open(img_path).convert('RGB')
    img_t = preprocess(img)
    img_t = img_t.unsqueeze(0)
    img_t = img_t.to(device)
    with torch.no_grad():
        emb = model(img_t)
    emb = emb / emb.norm(p=2, dim=1, keepdim=True)
    return emb.cpu().squeeze()

In [65]:
embedding = get_embedding("img/amir.jpg")
embedding1 = get_embedding("img/download.png")
embedding2 = get_embedding("img/amir1.jpg")
sag = get_embedding("img/sag.jpg")

c:\Users\PARATCO\Projects\face\models\arcface.py:149: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.fp16):


In [56]:
def cosinesim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
cosinesim(embedding1, embedding)

0.01134903991243969